# Packages 

In [29]:
%load_ext autoreload
%autoreload 2
import sys
import logging
sys.path.append('../')
import os
from utils import *

import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec

from annoy import AnnoyIndex
import polars as pl
import implicit
import scipy.sparse as sps
from utils import str2list


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Config 

In [22]:
debug = True

if debug:
    SAMPLE_NUM = 1000
else:
    SAMPLE_NUM = None

candidate_path = '../data/candidates/'
# train_data_dir = '.'
# test_data_dir = '.'
# task = 'task1'
# PREDS_PER_SESSION = 100

# num_tree = 100

# model_dir = '../model_training/als_v1'

# # target locales: locales needed for task1
# target_locals = ["DE", 'JP', 'UK']

# submit_file = f'submission_{task}_ALS.parquet'

# Function 

In [38]:
data_type = 'train'

def get_all_candidates(data_type):
    if data_type == 'train':
        w2v_file = os.path.join(candidate_path, 'task1_train_w2c_top100_300k.parquet')
        nic_file = os.path.join(candidate_path, 'task1_train_nic_without_pupular_top100_300k.parquet')
    else:
        w2v_file = os.path.join(candidate_path, 'task1_train_w2c_top100_300k.parquet')
        nic_file = os.path.join(candidate_path, 'task1_train_nic_without_pupular_top100_300k.parquet')
    w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM).with_columns(pl.col('prev_items').apply(str2list))
    nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
    print(nic_pl.schema)
    cols_to_keep = ['prev_items', 'next_item', 'locale', 'next_item_prediction', 'cg_source']

    w2v_pl = w2v_pl.with_columns(pl.lit('word2vec').alias('cg_source')).select(cols_to_keep)
    nic_pl = nic_pl.with_columns(pl.lit('next_item_counter').alias('cg_source')).select(cols_to_keep)

    cg_pl = (
            pl.concat([w2v_pl, nic_pl], how='vertical')
                .explode('next_item_prediction')
                .groupby(['prev_items', 'locale', 'next_item', 'next_item_prediction'])
                .agg(
                    pl.col('cg_source')
                )
                .with_columns(
                    pl.when(pl.col('cg_source').arr.contains(pl.lit('word2vec'))).then(1).otherwise(0).alias('whether_w2v')
                    , pl.when(pl.col('cg_source').arr.contains(pl.lit('next_item_counter'))).then(1).otherwise(0).alias('whether_nic')
                    # , pl.col('prev_items').arr.get(-1).alias('last_prev_item')

                )
    )
    if data_type == 'train':
        cg_pl = cg_pl.with_columns(
            pl.when(pl.col('next_item')==pl.col('next_item_prediction')).then(1).otherwise(0).alias('label')
        )
    return cg_pl

# Combine All candidates 

In [39]:
train_candidates = get_all_candidates(data_type='train')

{'prev_items': Utf8, 'next_item': Utf8, 'locale': Utf8, 'next_item_prediction': List(Utf8), 'rec_num': UInt32}


In [36]:
train_candidates.head().collect()

ShapeError: unable to vstack, dtypes for column "prev_items" don't match: `list[str]` and `str`

In [20]:
train_candidates.collect().shape

SchemaError: invalid series dtype: expected `List`, got `str`

In [12]:
train_candidates.select('label').mean().collect()

label
f64
0.00846


# Feature Process 

## Load model 

In [ ]:
nic_model = pl.scan_parquet('../model_training/next_item_counter_v2/nic_model.parquet').explode(['next_item_prediction', 'next_item_weight'])

In [ ]:
nic_model.schema

In [ ]:
train_candidates.schema

In [ ]:
def add_nic_feature(df, nic_model):
    df.join(nic_model, how='left',
            left_on=['last_prev_item', 'next_item_prediction']
           , right_on=)

In [15]:
# w2v_pl.with_columns(pl.lit('w2v').alias('cg_source')).select(cols_to_keep).head(3).collect()

In [16]:
w2v_pl.schema

{'prev_items': Utf8,
 'next_item': Utf8,
 'locale': Utf8,
 'next_item_prediction': List(Utf8),
 'cg_source': Utf8}

In [17]:
nic_pl.schema

{'prev_items': Utf8,
 'next_item': Utf8,
 'locale': Utf8,
 'next_item_prediction': List(Utf8),
 'cg_source': Utf8}

In [18]:
pl.concat([w2v_pl.collect(), nic_pl.collect()], how='horizontal')

DuplicateError: unable to hstack, column with name "prev_items" already exists